In [12]:
"""
Simple tester for the vgg19_trainable
"""
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf

import vgg16_2classes as vgg16
import os
import time
import numpy as np
from IPython.display import clear_output
import sys

PRETRAINED_MODEL_PATH= "/home/sik4hi/ckpt_dir"
N_EPOCHS = 100
INIT_LEARNING_RATE = 0.01
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 224    #960
IMAGE_WIDTH   = 224    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 100
N_CLASSES = 1000
DROPOUT = 0.50
ckpt_dir = "/home/sik4hi/ckpt_dir"
LOGS_PATH = '/home/sik4hi/tensorflow_logs'
WEIGHT_PATH = '.npy'
TRAINSET_PATH0 = '/mnt/data1/imagenet-data/csv-files/train2/imagenetdataall0.csv'
TRAINSET_PATH1 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata1.csv'
TRAINSET_PATH2 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata2.csv'
TRAINSET_PATH3 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata3.csv'
TRAINSET_PATH4 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata4.csv'
TRAINSET_PATH5 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata5.csv'
TRAINSET_PATH6 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata6.csv'
TRAINSET_PATH7 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata7.csv'
TRAINSET_PATH8 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata8.csv'
TRAINSET_PATH9 = '/mnt/data1/imagenet-data/csv-files/train/imagenetdata9.csv'

VALSET_PATH0 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata0.csv'
VALSET_PATH1 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata1.csv'
VALSET_PATH2 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata2.csv'
VALSET_PATH3 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata3.csv'
VALSET_PATH4 ='/mnt/data1/imagenet-data/csv-files/val/imagenetdata4.csv'

os.environ["CUDA_VISIBLE_DEVICES"] = ""
f=open(TRAINSET_PATH0,"r")
filepaths=[]
labels=[]
for line in f:
    filepath, label= line.split(",")
    label= int(label)
    filepaths.append(filepath)
    labels.append(label)

with tf.device('/cpu:0'):
    one_ex = tf.train.slice_input_producer([filepaths,labels#, TRAINSET_PATH1
                                              #, TRAINSET_PATH2, TRAINSET_PATH3,
                                              # TRAINSET_PATH4, TRAINSET_PATH5
                                              #, TRAINSET_PATH6, TRAINSET_PATH7, 
                                              # TRAINSET_PATH8, TRAINSET_PATH9
                                              ],capacity = 1281144, shuffle=True)
    
    
    im_content = tf.read_file(one_ex[0])
    train_image = tf.image.decode_jpeg(im_content, channels=3)
    
    # train_image = augment(train_image)
    #size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH], tf.int32)
    #train_image = tf.image.resize_images(train_image, size)
    train_label = tf.cast(one_ex[1], tf.int64) # unnecessary
    #train_label = one_ex[1]
    def _compute_longer_edge(height, width, new_shorter_edge):
        return tf.cast(width*new_shorter_edge/height, tf.int32)

    shape = tf.shape(train_image)
    height = shape[0]
    width = shape[1]
    new_shorter_edge = tf.constant(256, dtype=tf.int32)

    height_smaller_than_width = tf.less_equal(height, width)
    new_height_and_width = tf.cond(
        height_smaller_than_width,
        lambda: (new_shorter_edge, _compute_longer_edge(height, width, new_shorter_edge)),
        lambda: (_compute_longer_edge(width, height, new_shorter_edge), new_shorter_edge)
    )
    size = tf.cast([new_height_and_width[0], new_height_and_width[1]], tf.int32)
    train_image = tf.image.resize_images(train_image, size)
    size = tf.cast([IMAGE_HEIGHT, IMAGE_WIDTH, 3], tf.int32)
    train_image = tf.random_crop(train_image, size)
    train_image = tf.image.random_flip_left_right(train_image)
    train_image = tf.cast(train_image, tf.float32)/255. # necessary for mapping rgb channels from 0-255 to 0-1 float.
    #train_label_batch = tf.one_hot(train_label_batch, 1000)
    train_image_batch, train_label_batch = tf.train.shuffle_batch([train_image, train_label], batch_size=BATCH_SIZE,
                                                                  capacity = 1000 + 3 * BATCH_SIZE, 
                                                                  min_after_dequeue = 1000)
with tf.device('/cpu:0'):
    sess = tf.Session()
    init_op = tf.group(tf.initialize_all_variables(),
                       tf.initialize_local_variables())
    sess.run(init_op)
    #print(csv_path)
    # For populating queues with batches, very important!
    threads = tf.train.start_queue_runners(sess=sess)
    print("done")
    

done


In [13]:
epoch_start_time = time.time()
train_imbatch, train_labatch = sess.run([train_image_batch, train_label_batch])
print(train_labatch)
print 'Time Elapsed for Epoch:'+ ' is ' + str(
            (time.time() - epoch_start_time) / 60.) + ' minutes'

[868  21 797 800 495 556 746 720  91 451 527 784 362 437 919 690 397 461
 565 167 757 680 543 130 855 165 129 632 868 147 284 887 323 685 675 962
 856 665 108 783 476 170 336 780 462 207 936 892 861 312 903   1 341 299
 334 815  75 565 875 776 779 433 803 803 758 200 949 410 430 759 879 739
 972 537 371 640 704 795   1 954 771 838 337 414 411 424 688 661 386 793
 702 874 982 626 392 898 903  35 855 766]
Time Elapsed for Epoch: is 0.447953414917 minutes


In [4]:
TRAINSET_PATH0 = '/mnt/data1/imagenet-data/csv-files/train2/imagenetdataall0.csv'
f=open(TRAINSET_PATH0,"r")
filepaths=[]
labels=[]
for line in f:
    filepath, label= line.split(",")
    label= int(label)
    filepaths.append(filepath)
    labels.append(label)

937
